## Poker Hands

In the card game poker, a hand consists of five cards and are ranked, from lowest to highest, in the following way:

* High Card: Highest value card.
* One Pair: Two cards of the same value.
* Two Pairs: Two different pairs.
* Three of a Kind: Three cards of the same value.
* Straight: All cards are consecutive values.
* Flush: All cards of the same suit.
* Full House: Three of a kind and a pair.
* Four of a Kind: Four cards of the same value.
* Straight Flush: All cards are consecutive values of same suit.
* Royal Flush: Ten, Jack, Queen, King, Ace, in same suit.

The cards are valued in the order:
2, 3, 4, 5, 6, 7, 8, 9, 10, Jack, Queen, King, Ace.

If two players have the same ranked hands then the rank made up of the highest value wins; for example, a pair of eights beats a pair of fives (see example 1 below). But if two ranks tie, for example, both players have a pair of queens, then highest cards in each hand are compared (see example 4 below); if the highest cards tie then the next highest cards are compared, and so on.

Consider the following five hands dealt to two players: (the five example hands aren't shown because the formatting of the text is tricky).

The file, poker.txt, contains one-thousand random hands dealt to two players. Each line of the file contains ten cards (separated by a single space): the first five are Player 1's cards and the last five are Player 2's cards. You can assume that all hands are valid (no invalid characters or repeated cards), each player's hand is in no specific order, and in each hand there is a clear winner.

How many hands does Player 1 win?

### Methodology

To help process the hands and convert them into something more manageable, I created a Card object to split each 'card' into its rank (the number value or value corresponding to a face card). Then using the ranks and suits of the hands, we can return the outcomes of the players' hands as a tuple and compare them to find a winner. When finding a winner, we can slightly abuse the fact each hand has a winner when checking who it is. 

In [127]:
from collections import Counter

In [ ]:
# read in poker hand file
with open('files/0054_poker.txt', 'r') as file:
    hands = file.readlines()

# remove indent character from each hand
hands = [line.strip() for line in hands]

In [90]:
class Card:
    '''
    When creating a card, we've already turned the hand into 
    player1's cards and player2's cards. 
    '''
    def __init__(self, pair:str):
        ranks = {'2' : 2, '3' : 3, '4' : 4, '5' : 5, '6' : 6, '7' : 7,
              '8' : 8, '9' : 9, 'T' : 10, 'J' : 11, 'Q' : 12, 'K' : 13, 'A' : 14}
        self.rank = ranks[pair[0]]
        self.suit = pair[1]

In [218]:
# helper function to determine if hand is straight
def is_straight(ranks: list) -> bool:
    '''
    ranks should be a sorted list (desc order) of the ranks
    of each card
    '''
    return ranks == list(range(max(ranks), min(ranks)-1, -1))

# helper function to determine is hand is a flush
def is_flush(suits: list) -> bool:
    return len(set(suits)) == 1

# helper function for cases where the hand might be a flush, straight,
# straight flush, or royal flush
def hand_outcome(ranks: list, suits: list) -> tuple:
    '''
    Determine the outcome of the hand, starting with Royal Flush and 
    working down to High Card. The function will return a tuple (no
    consistent length) to be compared in the wrapper function

    Inputs:
    ranks:list - rank of cards in descending order
    suits:list - suit of cards
    '''
    rank_tallies = Counter(ranks)
    rank_apps = rank_tallies.most_common()

    if is_straight(ranks) and is_flush(suits): # either Royal Flush or Straight Flush
        if max(ranks) == 14: # Royal Flush - can just return score
            return (9)
        else: # Straight Flush
            return (8, max(ranks))
    elif 4 in rank_tallies.values(): # Four of a Kind
        # extract rank of the four cards and rank of remaining card
        four_card = rank_apps[0][0] 
        other_card = rank_apps[1][0]
        return (7, four_card, other_card)
    elif (3 in rank_tallies.values()) and (2 in rank_tallies.values()): # Full House
        # extract rank of three of kind and rank of pair
        three_card = rank_apps[0][0]
        pair_card = rank_apps[1][0]
        return (6, three_card, pair_card)
    elif is_flush(suits): # Flush (not Straight or Royal)
        # tuple will include order of cards in flush
        return (5, rank_apps[0][0], rank_apps[1][0], rank_apps[2][0], rank_apps[3][0], rank_apps[4][0])
    elif is_straight(ranks): # Straight (not also a Flush)
        # extract highest card in straight
        return (4, max(ranks))
    elif (3 in rank_tallies.values()): # Three of a Kind
        # extract three of a kind rank and rank of other two cards (in order)
        three_card = rank_apps[0][0]
        return (3, three_card, rank_apps[1][0], rank_apps[2][0])
    elif rank_apps[0][1] == 2 and rank_apps[1][1] == 2: # Two Pairs
        # extract rank of higher pair, then rank of lower pair, and remaining card
        high_pair = rank_apps[0][0]
        low_pair = rank_apps[1][0]
        return (2, high_pair, low_pair, rank_apps[2][0])
    elif (2 in rank_tallies.values()): # Two of a Kind
        # extract pair rank and rank of three other cards (in order)
        pair_card = rank_apps[0][0]
        return (1, pair_card, rank_apps[1][0], rank_apps[2][0], rank_apps[3][0])
    else: # High Card
        # return a tuple with the cards in order
        return (0,) + tuple(rank_tallies.keys())

# wrapper function for evaluating hands
def find_winner(deal:str):
    '''
    deal:str - line from the file which includes the characters for both hands

    The goal of problem is counting how many times player 1 wins, thus the 
    function will return a 1 if P1 wins and 0 if P2 wins

    Additionally, the problem mentions each deal will have a winner. Thus when
    finding the winner can we iterate through the outcome tuple elements until one
    has a larger value than another. Either one tuple will have a higher first element 
    (i.e. their hand was higher in the hierarchy) or the have the same kind of hand and
    the function checks which hand has the better cards 
    '''
    player1 = deal[:14]
    player2 = deal[15:]

    # convert hands to list of tuples (rank first, suit second)
    p1_hand = [Card(card) for card in player1.split()]
    p2_hand = [Card(card) for card in player2.split()]

    # extract ranks and suits from both hands
    p1_ranks = sorted([c.rank for c in p1_hand], reverse = True)
    p1_suits = [c.suit for c in p1_hand]
    p2_ranks = sorted([c.rank for c in p2_hand], reverse = True)
    p2_suits = [c.suit for c in p2_hand]

    # get tuple for hand outcome of both players
    p1_outcome = hand_outcome(p1_ranks, p1_suits)
    p2_outcome = hand_outcome(p2_ranks, p2_suits)

    # find winner
    idx = 0
    while True:
        if p1_outcome[idx] > p2_outcome[idx]: # P1 wins
            return 1
        elif p1_outcome[idx] < p2_outcome[idx]: # P2 wins
            return 0
        else: # value is the same, iterate to next index
            idx += 1

In [222]:
# find how many games player 1 wins
total = 0
for hand in hands:
    total += find_winner(hand)
# print result
print(f'Number of hands Player 1 wins: {total}')

Number of hands Player 1 wins: 376
